In [ ]:
from CC import CC
from DnsVhost import DnsVhost
from MariaDB import MariaDB
from WP import WP
from Partition import Partition
from SMB import SMB
from NFS import NFS
from Diskquota import Diskquota
from ISCSI import ISCSI
from FTP import FTP
from RAID import RAID
from Tomcat import Tomcat
from Nginx import Nginx
from SNMP import SNMP
from Cacti import Cacti
from JL import JL

#---------------------------------------#
#             SSH 객체  생성             #
#---------------------------------------#
server_ip = "172.16.16.66"
server_account = "root"
server_passwd = "bs401!@"
cc =  CC(f"{server_ip}", f"{server_account}", f"{server_passwd}")
cc.connect()

#---------------------------------------#
#         여러 클래스 객체 생성            #
#---------------------------------------#

os = input("현재 OS를 입력해주세요. ex. Rocky, Ubuntu : ")

dv = DnsVhost(os) 
mariadb = MariaDB(os)
wp = WP(os)
part = Partition(os)
smb = SMB(os)
nfs = NFS(os)
dq = Diskquota(os)
ftp = FTP(os)
raid = RAID(os)
tomcat = Tomcat(os)
nginx = Nginx(os)
snmp = SNMP(os)
cacti = Cacti(os)
jl = JL(os)

#---------------------------------------#
#                 전역변수               #
#---------------------------------------#
ZONEFILE = ""


###################################################
#              HTTP,DNS -DnsVhost.py              #
###################################################
## 1. http, DNS 설치 및 초기 세팅 
def install_and_setting_http_dns() :
    print("[HTTP, DNS] 설치 시작합니다.")
    #========= HTTP ========#
    cc.run(dv.install_http())
    dv.set_servername()
    cc.run(dv.mk_vhostconf())
    cc.run(dv.test_vhost())
    # web_dir,third_domain,error_log ,custom_log
    cc.run(dv.set_vhostconf("/home/test","test"))

    #========= DNS ========#
    cc.run(dv.install_named())
    cc.run(dv.init_set_named_conf())
    cc.run(dv.init_set_named_zone())
    cc.run(dv.mk_zone_file())
    cc.run(dv.init_set_zone_file())
    cc.run(dv.set_dns_Third_domain("test"))

    #======= RESTART ========#
    cc.run(dv.restart_http())
    cc.run(dv.restart_dns())
    print("[HTTP, DNS] 설치 및 설정 완료")
    
## 2. 초기 세팅 이후에 새롭게 DnsVhost클래스 사용할 경우에 해당됨 (기본)
def mod_http_dns(web_dir,third_domain):
    print("[HTTP, DNS] 추가 시작")
    dv.set_servername()
    #set_vhostconf(self,web_dir,third_domain):
    cc.run(dv.set_vhostconf(web_dir,third_domain))
    cc.run(f"mkdir -p {web_dir} && chmod 777 {web_dir}")
    cc.run(dv.set_dns_Third_domain(third_domain))
    cc.run(dv.restart_dns())
    cc.run(dv.restart_http())
    print("[HTTP, DNS] 추가 완료")

## 3. 초기 세팅 이후에 새롭게 DnsVhost클래스 사용할 경우에 해당됨 (리버시 프록시)
def mod_http_dns_reverse_proxy(third_domain,port_num):
    print("[HTTP, DNS] 추가 시작 <리버시 프록시>")
    dv.set_servername()
    #set_reverse_proxy_vhostconf(self,third_domain,port_num)
    cc.run(dv.set_reverse_proxy_vhostconf(third_domain,port_num))
    cc.run(dv.set_dns_Third_domain(third_domain))
    cc.run(dv.restart_dns())
    cc.run(dv.restart_http())
    print("[HTTP, DNS] 추가 완료 <리버시 프록시>")
#---------------------실행부-----------------------#
#install_and_setting_http_dns()
#mod_http_dns("/home/test2/","test2")



###################################################
#            MariaDB,PHP -MariaDB.py              #
###################################################
## 1. mariaDB와 php 둘 다 설치함
def install_mariaDB_and_php():
    print("[mariaDB] 설치 시작합니다.")
    cc.run(mariadb.install_packages())
    cc.run(mariadb.install_apache_and_php())
    cc.run(mariadb.install_mariaDB())
    cc.run(mariadb.init_mariaDB())
    cc.run(mariadb.install_php_my_admin())
    cc.run(mariadb.set_phpMyadmin_conf())
    print("[mariaDB] 설치 완료")
#---------------------실행부-----------------------#
#install_mariaDB_and_php()


##################################################
#           Partition -Partition.py              #
##################################################
def mk_partition():
    print("[partition] 시작합니다.")
    cc.run("lsblk")
    cc.run(part.make_partition())
    cc.run(part.format_partition())
    cc.run(part.mount())
    cc.run(part.permanent_mount_config_remote())
    print("[partition] 완료")

#---------------------실행부-----------------------#
#mk_partition()

###################################################
#                    SMB -SMB.py                  #
###################################################

def install_smb():
    cc.run(smb.smb_install())
    cc.run(smb.smb_intsll_status_check())   
    choose = input("smb 서버와 클라이언트 중 선택하세요.(S/C) ex)S : ")
    if(choose == "S" ) :
        server_smb()
    if(choose == "C" ) :
        client_smb()

def server_smb():
    print("[SMB] SERVER 시작")
    cc.run(smb.smb_share_folder_make())
    cc.run(smb.smb_mk_smb_id())
    cc.run(smb.smb_set_smb_id_passwd())
    cc.run(smb.smb_id_check())
    cc.run(smb.smb_share_folder_authority())
    cc.run(smb.smb_mk_back())
    cc.run(smb.smb_set())
    cc.run(smb.smb_server_start())
    print("[SMB] SERVER 설정완료")    
def client_smb() :
    print("[SMB] CLIENT 시작")
    cc.run(smb.smb_client_login())
    cc.run(smb.smb_client_mount())
    print("[SMB] CLIENT 완료")
#---------------------실행부-----------------------#
#install_smb()

###################################################
#                    NFS -NFS.py                  #
###################################################

def install_nfs():
    cc.run(nfs.nfs_isntall())
    choose = input("NFS 서버와 클라이언트 중 선택하세요.(S/C) ex)S : ")
    if(choose == "S" ) :
        server_nfs()
    if(choose == "C" ) :
        client_nfs()

def server_nfs():
    cc.run(nfs.nfs_share_folder_make())
    cc.run(nfs.nfs_share_folder_authority())
    cc.run(nfs.server_nfs_set_exports_config())
    cc.run(nfs.server_nfs_start())

def client_nfs():
    cc.run(nfs.nfs_client_mount_temporary())
    cc.run(nfs.nfs_clinet_mount_permanent())
#---------------------실행부-----------------------#
#install_nfs()

###################################################
#            Diskquota -diskquota.py              #
###################################################
def diskquota_install_and_set():
    cc.run(dq.diskquota_isntall())
    cc.run(dq.diskquota_install_check())
    cc.run("lsblk")
    print("TYPE MOUNTPOINTS가 part로 되어있는 디스크만 가능합니다. 안되면 partition을 해야합니다")
    exist_part = input("part로 되어있는 디스크가 없으시거나 해당 디스크가 마운트가 안되어있는 경우에 반드시 n 입력해주세요.\n 있으시면 y 또는 아무문자열 입력 ex) y: ")
    if(exist_part == "n") :
        return
    cc.run(dq.diskquota_modify())
    cc.run(dq.diskquota_mount_and_initalization())
    cc.run(dq.diskquota_settings())
    cc.run(dq.diskquota_systemctl())
    
#---------------------실행부-----------------------#
#diskquota_install_and_set()



###################################################
#                  ISCSI -ISCSI.py                #
###################################################
def iscsi_all_in_one():
    client_os = input("클라이언의 OS를 입력하세요 ex. Rocky, Ubuntu : ")
    iscsi = ISCSI(os,client_os)

    #=========== server 영역 ===========#
    cc.run(iscsi.set_firewall())
    cc.run(iscsi.install_target())
    
    #=========== client 영역 ===========#
    client_ip = input("새로운 SSH 연결을 합니다. 클라이언트 IP를 입력하세요")
    client_account = input("클라이언트 계정 입력하세요 ex. root")
    client_passwd = input("클라이언트 계정의 비밀번호를 입력하세요 ex. asd123!@")
    cc_client =  CC(f"{client_ip}", f"{client_account}", f"{client_passwd}")
    cc_client.connect()
    
    cc_client.run(iscsi.iscsi_client_install())
    cc_client.run(iscsi.iscsi_client_iqn())
    iscsi_client_iqn_domain = input("위에서 확인 CLIENT의 IQN을 복사해주세요")

    #=========== server 영역 ===========#
    print("[ISCSI 서버] ISCSI 서버를 세팅해보겠습니다.")
    cc.run("lsblk")
    disk_path = input("위의 결과를 확인하시고 iSCSI로 제공할 디스크 경로를 입력하세요 : ex)/dev/sdb")
    cc.run(iscsi.targetcli(server_ip, iscsi_client_iqn_domain, disk_path))

    #=========== client 영역 ===========#
    print("[ISCSI 클라이언트] ISCSI 클라이언트를 세팅해보겠습니다.")
    cc_client.run(iscsi.iscsi_client_login_set_servers_iscsi(server_ip))
    cc_client.run("iscsiadm -m session")
    print("이제 클라이언트로 돌아가셔서 lsblk로 확인해보세요. 또 iscsiadm -m session로 잘 받았는지 확인해보세요 ^^")
    print("[ISCSI] 모든 설정 및 세팅이 완료되었습니다.")
#---------------------실행부-----------------------#
#iscsi_all_in_one()

###################################################
#                    FTP -FTP.py                  #
###################################################
def inital_FTP():
    print("[FTP 서버] FTP 설치 및 유저를 생성합니다.")
    cc.run(ftp.install_ftp())
    cc.run(ftp.on_off_ftp())
    cc.run(ftp.set_ftp_config())
    cc.run(ftp.add_ftpuser())
    cc.run(ftp.set_ftpuser_passwd())
    cc.run(ftp.set_dir())
    print("[FTP 서버] 설치 및 설정이 완료되었습니다. ")
#---------------------실행부-----------------------#
#inital_FTP()

###################################################
#                  RAID -RAID.py                  #
###################################################
def mk_raid1():
    print("[RAID] RAID1을 만듭니다.")
    cc.run(raid.install_raid())
    cc.run("lsblk")
    cc.run(raid.raid1_mk())
    cc.run(raid.raid1_mk_fs_mount())
    permanent_mount = input("영구마운트 하겠습니까 yes or no (ex. yes) : ")    
    if(permanent_mount == "yes") :
        cc.run(raid.raid1_permanent_mount_config_remote())
    cc.run("lsblk")
    print("[RAID] RAID1 완료")
    
def erase_raid():
    print("[RAID] RAID를 제거합니다.")
    cc.run("lsblk")
    cc.run(raid.erase_raid())
    cc.run("lsblk")
    print("RAID가 제거되었습니다 ^^")
    
#---------------------실행부-----------------------#
#mk_raid1()
#erase_raid()


###################################################
#               Tomcat -Tomcat.py                 #
###################################################
def install_tomcat():
    print("[Tomcat] 설치하겠습니다.")
    cc.run(tomcat.install_java())
    cc.run(tomcat.set_tomcat())
#    mod_http_dns_reverse_proxy(third_domain,"port_num"):
    mod_http_dns_reverse_proxy("tom","8088")
    cc.run(tomcat.set_service())
    print("[Tomcat] 설치 및 설정이 완료되었습니다.")

#---------------------실행부-----------------------#
#install_tomcat()

###################################################
#                 WP,PHPMYSQL -WP.py              #
###################################################
## 1. 로컬) WP 설치 및 wp-config.php 파일 수정까지
def install_wp():
    print("[WP] 설치하겠습니다.")
    print("[전제조건] mariaDB가 있어야하며, 서버팀에게 wp를 위한 유저 및 db를 생성해야합니다.")
    cc.run(wp.install_prev_packages())
    cc.run(wp.install_and_move_dir())
    cc.run(wp.set_wp_config_php_client())
    #def mod_http_dns(web_dir,third_domain):
    mod_http_dns("/home/wp","wp")
    print("[WP] 설치 및 설정이 완료되었습니다.")
#---------------------실행부-----------------------#
#install_wp()

###################################################
#                  Nginx -Nginx.py                #
###################################################
## 1. 로컬) WP 설치 및 wp-config.php 파일 수정까지
def install_nginx():
    print("[Nginx] 설치하겠습니다.")
    print("[전제조건] httpd, dns, php, wp(시범용)가 있어야합니다. ")
    cc.run(nginx.install_nginx())
    servername = input("server name 입력 ex. test.com , ppp.com : ")
    cc.run(nginx.set_vhostconf(servername))
    cc.run(nginx.restart_nginx())
    print("[Nginx] 설치 및 설정이 완료되었습니다.")
#---------------------실행부-----------------------#
#install_nginx()


###################################################
#                    SNMP -SNMP.py                #
###################################################
## 1. 로컬) WP 설치 및 wp-config.php 파일 수정까지
def install_snmp():
    print("[SNMP] 설치하겠습니다.")
    print("[전제조건] Rocky는 dnf있어야함. ")
    cc.run(snmp.install_snmp())
    cc.run(snmp.set_snmp())
    cc.run(snmp.start_enable_snmpd())
    cc.run(snmp.test_snmpd_local())
    cc.run(snmp.test_snmpd_public())
    print("[SNMP] 설정 완료")
    
#---------------------실행부-----------------------#
#install_snmp()

###################################################
#                  Cacti -Cacti.py                #
###################################################
## 1. 로컬) WP 설치 및 wp-config.php 파일 수정까지
def install_cacti():
    print("[Cacti] 설치하겠습니다.")
    print("[전제조건] MRTG 있어야함. ")
    cc.run(cacti.install_cacti())
    cc.run(cacti.set_mariadb_server_cnf())
    cc.run(cacti.set_cacti())
    cc.run(cacti.set_cacti_db_php())
    cc.run(cacti.set_cacti_conf())
    cc.run(cacti.restart_all())

    print("#======= timezone 세팅 시작합니다. =========#")
    cc.run(cacti.set_my_cnf())
    cc.run(cacti.set_timezone())
    cc.run(cacti.grant_timezone_to_user())
    cc.run(cacti.set_php_ini())
    cc.run(cacti.restart_all())

    print("#======= cron.d 세팅 시작합니다. =========#")
    cc.run(cacti.set_crond())
    
    print("[Cacti] 설정 완료")
    print("이제 브라우저에 접속해주세요.") 
    print("1. [초기 세팅] username : admin , password : admin 입니다.")
    print("2. [Change Password] current password : admin , New password : Asd123!@ 입니다.")
    
#---------------------실행부-----------------------#
#install_cacti()

###################################################
#                    JL -JL.py                    #
###################################################
## 1. 로컬) WP 설치 및 wp-config.php 파일 수정까지
def install_JL():
    print("[JL] 설치하겠습니다.")
    print("[전제조건] MariaDB JL용 계정 및 JL DB 있어야함. ")
    cc.run(jl.install_pacakages())
    cc.run(jl.install_JL())
#def mod_http_dns(web_dir,third_domain):
    mod_http_dns("/home/jl","jl")
    print("[JL] 설정 완료")

#---------------------실행부-----------------------#
#install_JL()

###################################################
#                  DHCP -DHCP.py                  #
###################################################
## 1. 로컬) WP 설치 및 wp-config.php 파일 수정까지
def install_DHCP():
    print("[DHCP] 기능 구현중입니다.. ㅠㅠ")

#---------------------실행부-----------------------#
#install_JL()

###################################################
#             FIREWALL -FIREWALL.py               #
###################################################
## 1. 로컬) WP 설치 및 wp-config.php 파일 수정까지
def install_Firewall():
    print("[Firewall] 기능 구현중입니다.. ㅠㅠ")
#---------------------실행부-----------------------#
#install_JL()

#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@#
#                       구현부                      #
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@#

menu = {
    "1": inital_FTP,
    "2": install_mariaDB_and_php,
    "3": install_and_setting_http_dns,
    "4": install_DHCP,
    "5": install_Firewall,
    "6": install_nfs,
    "7": install_smb,
    "8": mk_partition,
    "9": mk_raid1,
    "10": iscsi_all_in_one,
    "11": diskquota_install_and_set,
    "12": install_wp,
    "13": install_JL,
    "14": install_nginx,
    "15": install_tomcat,
    "16": install_snmp,
    "17": install_cacti
}

while True:
    print("""
1.  FTP
2.  MariaDB
3.  DnsVhost
4.  DHCP
5.  Firewall
6.  NFS
7.  SMB
8.  PARTITION
9.  RAID
10. ISCSI
11. Diskquota
12. WP
13. JL
14. Nginx
15. Tomcat
16. SNMP (Rocky only)
17. Cacti (Rocky only)
0.  종료
""")

    choice = input("번호 선택: ").strip()

    if choice == "0":
        print("종료합니다.")
        break

    if choice in menu:
        menu[choice]()   # ⭐ 여기서 함수 실행
    else:
        print("잘못된 번호입니다.")



실행


현재 OS를 입력해주세요. ex. Rocky, Ubuntu :  Rocky


MariaDB init start
현재 당신의 os는 Rocky입니다.

1.  FTP
2.  MariaDB
3.  DnsVhost
4.  DHCP
5.  Firewall
6.  NFS
7.  SMB
8.  PARTITION
9.  RAID
10. ISCSI
11. Diskquota
12. WP
13. JL
14. Nginx
15. Tomcat
16. SNMP (Rocky only)
17. Cacti (Rocky only)
0.  종료



번호 선택:  16


[SNMP] 설치하겠습니다.
[전제조건] Rocky는 dnf있어야함. 
Rocky Linux 9 - BaseOS                          5.7 kB/s | 4.3 kB     00:00    
Rocky Linux 9 - AppStream                       8.2 kB/s | 4.8 kB     00:00    
Rocky Linux 9 - Extras                          5.3 kB/s | 3.1 kB     00:00    
Dependencies resolved.
 Package                      Arch    Version                  Repository  Size
Installing:
 net-snmp                     x86_64  1:5.9.1-17.el9           appstream  295 k
 net-snmp-agent-libs          x86_64  1:5.9.1-17.el9           appstream  693 k
 net-snmp-devel               x86_64  1:5.9.1-17.el9           appstream  217 k
 net-snmp-libs                x86_64  1:5.9.1-17.el9           appstream  740 k
 net-snmp-perl                x86_64  1:5.9.1-17.el9           appstream  302 k
 net-snmp-utils               x86_64  1:5.9.1-17.el9           appstream  186 k
Installing dependencies:
 annobin                      x86_64  12.98-1.el9              appstream  1.1 M
 cpp         

74번째 줄 : 설정할 local의 COMMUNITY 이름을 입력해주세요 ex. kong :  kong
74번째 줄 : 설정할 mynetwork의 COMMUNITY 이름을 입력해주세요 ex. tj :  tj
75번째 줄 : 설정할 mynetwork의 대역을 입력해주세요. ex. 172.16.0.0/16 :  172.16.0.0/16



Created symlink /etc/systemd/system/multi-user.target.wants/snmpd.service → /usr/lib/systemd/system/snmpd.service.

자기자신에게 테스트합니다. rocky인 경우만
SNMPv2-MIB::sysDescr.0 = STRING: Linux localhost.localdomain 5.14.0-611.16.1.el9_7.x86_64 #1 SMP PREEMPT_DYNAMIC Mon Dec 22 12:21:56 UTC 2025 x86_64
SNMPv2-MIB::sysObjectID.0 = OID: NET-SNMP-MIB::netSnmpAgentOIDs.10
DISMAN-EVENT-MIB::sysUpTimeInstance = Timeticks: (43) 0:00:00.43
SNMPv2-MIB::sysContact.0 = STRING: Root <root@localhost> (configure /etc/snmp/snmp.local.conf)
SNMPv2-MIB::sysName.0 = STRING: localhost.localdomain
SNMPv2-MIB::sysLocation.0 = STRING: Unknown (edit /etc/snmp/snmpd.conf)
SNMPv2-MIB::sysORLastChange.0 = Timeticks: (0) 0:00:00.00
SNMPv2-MIB::sysORID.1 = OID: SNMP-FRAMEWORK-MIB::snmpFrameworkMIBCompliance
SNMPv2-MIB::sysORID.2 = OID: SNMP-MPD-MIB::snmpMPDCompliance
SNMPv2-MIB::sysORID.3 = OID: SNMP-USER-BASED-SM-MIB::usmMIBCompliance
SNMPv2-MIB::sysORID.4 = OID: SNMPv2-MIB::snmpMIB
SNMPv2-MIB::sysORID.5 = OID: SNMP-VIEW-BA